In [1]:
import pandas
import researchpy as rp
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp


# Analysis <font color=blue>after </font> combining RF, DT and SVM into 1C

In [2]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['dataset', 'time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "1c")
df.methods = df.methods.replace("['decision_tree']", "1c")
df.methods = df.methods.replace("['random_forest']", "1c")
fsIs_b = ['vowel', 'openml_phpJNxH0q', 'dataset_31_credit-g', 'dataset_40_sonar']
fsIs_m = ['solar-flare_1', 'wine-quality-red', 'dataset_39_ecoli', 'synthetic_control']
fsIl_b = ['AirlinesCodrnaAdult', 'MagicTelescope', 'electricity-normalized', 'phpmPOD5A']
fsIl_m = ['pokerhand-normalized', 'eye_movements', 'avila-tr']
flIs_b = ['audiology', 'arrhythmia', 'AP_Breast_Lung', 'AP_Omentum_Ovary']
flIs_m = ['Amazon', 'umistfacescropped', 'phpGUrE90']
flIl_b = ['gina_agnostic', 'hiva_agnostic', 'phpZrCzJR', 'phprAeXmK']
flIl_m = ['KDDCup99', 'connect-4', 'dataset_60_waveform-5000', 'dataset_186_satimage']
df_multi = df[df.dataset.isin(fsIs_m) | df.dataset.isin(fsIl_m) | df.dataset.isin(flIs_m) | df.dataset.isin(flIl_m)]
df = df_multi
#df_multi = df[df[(((df.dataset in fsIs_m) | (df.dataset in fsIl_m)) | ((df.dataset in flIs_m) | (df.dataset in flIl_m)))].index]
#df.drop(df[(df.methods=='fc') & ((df.time_budget==30) | (df.time_budget==10))].index, inplace=True)
#df.drop(df[(df.methods=='3c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.shape

(630, 4)

In [3]:
rp.summary_cont(df['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,630.0,0.812905,0.173204,0.006901,0.799354,0.826456


In [4]:
rp.summary_cont(df.groupby(['time_budget']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
time_budget,,,,,,
10,210,0.793691,0.200539,0.013838,0.766568,0.820814
30,210,0.821646,0.156562,0.010804,0.800471,0.842821
60,210,0.823378,0.158115,0.010911,0.801992,0.844763


In [5]:
rp.summary_cont(df.groupby(['methods']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
methods,,,,,,
1c,378,0.791418,0.189042,0.009723,0.772360,0.810476
3c,126,0.840530,0.147077,0.013103,0.814849,0.866211
fc,126,0.849741,0.134202,0.011956,0.826308,0.873174


In [6]:
rp.summary_cont(df.groupby(['time_budget', 'methods']))['f1score']

N      Mean        SD        SE  95% Conf.  Interval
time_budget methods                                                        
10          1c       126  0.764683  0.224440  0.019995   0.725493  0.803872
            3c        42  0.834210  0.155150  0.023940   0.787288  0.881133
            fc        42  0.840197  0.144469  0.022292   0.796504  0.883889
30          1c       126  0.804258  0.166165  0.014803   0.775244  0.833273
            3c        42  0.855280  0.133299  0.020568   0.814966  0.895594
            fc        42  0.840175  0.143396  0.022127   0.796807  0.883543
60          1c       126  0.805313  0.169463  0.015097   0.775722  0.834903
            3c        42  0.832099  0.154185  0.023791   0.785468  0.878730
            fc        42  0.868852  0.113642  0.017535   0.834483  0.903222

In [7]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model = ols('f1score ~ C(time_budget)*C(methods)', df).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

Overall model F( 8, 621) =  2.609, p =  0.0082


In [8]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     2.609
Date:                Sun, 03 Nov 2019   Prob (F-statistic):            0.00820
Time:                        17:08:41   Log-Likelihood:                 221.55
No. Observations:                 630   AIC:                            -425.1
Df Residuals:                     621   BIC:                            -385.1
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                 0.7647      0.015     50.062      0.000       0.735       0.795
C(time_budget)[T.30]                      0.0396      0.022      1.832      0.067      -0.003       0.082
C(time_budget)[T.60]                      0.0406      0.022      1.881      0.060      -0.002       0.083
C(methods)[T.3c]                          0.0695      0.031      2.276      0.023       0.010       0.130
C(methods)[T.fc]                          0.0755      0.031      2.472      0.014       0.016       0.136
C(time_budget)[T.30]:C(methods)[T.3c]    -0.0185      0.043     -0.428      0.669      -0.103       0.066
C(time_budget)[T.60]:C(methods)[T.3c]    -0.0427      0.043     -0.989      0.323      -0.128       0.042
C(time_budget)[T.30]:C(methods)[T.fc]    -0.0396      0.043     -0.917      0.360      -0.124       0.045
C(time_budget)[T.60]:C(methods)[T.fc]    -0.0120      0.043     -0.277      0.782      -0.097       0.073
==============================================================================
Omnibus:                      119.962   Durbin-Watson:                   0.574
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              224.466
Skew:                          -1.101   Prob(JB):                     1.81e-49
Kurtosis:                       4.925   Cond. No.                         11.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
res

,sum_sq,df,F,PR(>F)
C(time_budget),0.116605,2.0,1.983212,0.138497
C(methods),0.441647,2.0,7.511525,0.000598
C(time_budget):C(methods),0.055367,4.0,0.470838,0.757170
Residual,18.256117,621.0,NaN,NaN


In [10]:
# Fits the model
model2 = ols('f1score ~ C(time_budget)+ C(methods)', df).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

Overall model F( 4, 625) =  4.764, p =  0.0009


In [11]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     4.764
Date:                Sun, 03 Nov 2019   Prob (F-statistic):           0.000861
Time:                        17:08:41   Log-Likelihood:                 220.60
No. Observations:                 630   AIC:                            -431.2
Df Residuals:                     625   BIC:                            -409.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.7722      0.013     59.135      0.000       0.747       0.798
C(time_budget)[T.30]     0.0280      0.017      1.674      0.095      -0.005       0.061
C(time_budget)[T.60]     0.0297      0.017      1.777      0.076      -0.003       0.062
C(methods)[T.3c]         0.0491      0.018      2.789      0.005       0.015       0.084
C(methods)[T.fc]         0.0583      0.018      3.312      0.001       0.024       0.093
==============================================================================
Omnibus:                      124.181   Durbin-Watson:                   0.573
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              240.270
Skew:                          -1.120   Prob(JB):                     6.70e-53
Kurtosis:                       5.034   Cond. No.                         3.96
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

,sum_sq,df,F,PR(>F)
C(time_budget),0.116605,2.0,1.989951,0.137567
C(methods),0.441647,2.0,7.537050,0.000583
Residual,18.311484,625.0,NaN,NaN


In [13]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(time_budget),0.116605,0.058302,2.0,1.989951,0.137567,0.006179,0.003069
C(methods),0.441647,0.220823,2.0,7.537050,0.000583,0.023405,0.020268
Residual,18.311484,0.029298,625.0,NaN,NaN,NaN,NaN


In [14]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    1c     3c   0.0491 0.0152  0.0077 0.0905   True
    1c     fc   0.0583 0.0029  0.0169 0.0998   True
    3c     fc   0.0092 0.8999 -0.0415   0.06  False
---------------------------------------------------


In [15]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['time_budget'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    10     30    0.028 0.2233 -0.0117 0.0676  False
    10     60   0.0297 0.1846   -0.01 0.0693  False
    30     60   0.0017    0.9 -0.0379 0.0414  False
---------------------------------------------------


# Analysis <font color=blue>without </font> combining RF, DT and SVM into 1C

In [16]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['dataset', 'time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "svm")
df.methods = df.methods.replace("['decision_tree']", "DT")
df.methods = df.methods.replace("['random_forest']", "RF")
fsIs_b = ['vowel', 'openml_phpJNxH0q', 'dataset_31_credit-g', 'dataset_40_sonar']
fsIs_m = ['solar-flare_1', 'wine-quality-red', 'dataset_39_ecoli', 'synthetic_control']
fsIl_b = ['AirlinesCodrnaAdult', 'MagicTelescope', 'electricity-normalized', 'phpmPOD5A']
fsIl_m = ['pokerhand-normalized', 'eye_movements', 'avila-tr']
flIs_b = ['audiology', 'arrhythmia', 'AP_Breast_Lung', 'AP_Omentum_Ovary']
flIs_m = ['Amazon', 'umistfacescropped', 'phpGUrE90']
flIl_b = ['gina_agnostic', 'hiva_agnostic', 'phpZrCzJR', 'phprAeXmK']
flIl_m = ['KDDCup99', 'connect-4', 'dataset_60_waveform-5000', 'dataset_186_satimage']
df_binary = df[df.dataset.isin(fsIs_b) | df.dataset.isin(fsIl_b) | df.dataset.isin(flIs_b) | df.dataset.isin(flIl_b)]
df = df_binary
#df.drop(df[(df.methods=='fc') & ((df.time_budget==30) | (df.time_budget==10))].index, inplace=True)
#df.drop(df[(df.methods=='3c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.shape

(720, 4)

In [17]:
rp.summary_cont(df['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,720.0,0.803741,0.210927,0.007861,0.788309,0.819174


In [18]:
rp.summary_cont(df.groupby(['time_budget']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
time_budget,,,,,,
10,240,0.796628,0.229231,0.014797,0.767626,0.825629
30,240,0.810851,0.203232,0.013119,0.785139,0.836564
60,240,0.803745,0.199726,0.012892,0.778476,0.829014


In [19]:
rp.summary_cont(df.groupby(['methods']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
methods,,,,,,
3c,144,0.826060,0.170904,0.014242,0.798145,0.853974
DT,144,0.799846,0.184425,0.015369,0.769723,0.829968
RF,144,0.818438,0.188115,0.015676,0.787712,0.849163
fc,144,0.832455,0.166686,0.013891,0.805230,0.859681
svm,144,0.741908,0.303130,0.025261,0.692397,0.791420


In [20]:
rp.summary_cont(df.groupby(['time_budget', 'methods']))['f1score']

N      Mean        SD        SE  95% Conf.  Interval
time_budget methods                                                       
10          3c       48  0.825507  0.176617  0.025492   0.775542  0.875472
            DT       48  0.795051  0.199168  0.028747   0.738706  0.851396
            RF       48  0.801832  0.235667  0.034016   0.735161  0.868502
            fc       48  0.829852  0.174626  0.025205   0.780450  0.879254
            svm      48  0.730896  0.322321  0.046523   0.639711  0.822081
30          3c       48  0.828860  0.162026  0.023386   0.783022  0.874697
            DT       48  0.806209  0.182877  0.026396   0.754473  0.857945
            RF       48  0.832476  0.168787  0.024362   0.784726  0.880226
            fc       48  0.838473  0.159629  0.023041   0.793314  0.883633
            svm      48  0.748238  0.301248  0.043481   0.663014  0.833462
60          3c       48  0.823813  0.177222  0.025580   0.773676  0.873949
            DT       48  0.798277  0.174084  0.025127   0.749028  0.847525
            RF       48  0.821006  0.152152  0.021961   0.777962  0.864050
            fc       48  0.829040  0.168830  0.024369   0.781278  0.876803
            svm      48  0.746591  0.291111  0.042018   0.664235  0.828947

In [21]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model = ols('f1score ~ C(time_budget)*C(methods)', df).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

Overall model F( 14, 705) =  1.312, p =  0.1938


In [22]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.312
Date:                Sun, 03 Nov 2019   Prob (F-statistic):              0.194
Time:                        17:08:42   Log-Likelihood:                 108.62
No. Observations:                 720   AIC:                            -187.2
Df Residuals:                     705   BIC:                            -118.6
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                  0.8255      0.030     27.197      0.000       0.766       0.885
C(time_budget)[T.30]                       0.0034      0.043      0.078      0.938      -0.081       0.088
C(time_budget)[T.60]                      -0.0017      0.043     -0.039      0.969      -0.086       0.083
C(methods)[T.DT]                          -0.0305      0.043     -0.710      0.478      -0.115       0.054
C(methods)[T.RF]                          -0.0237      0.043     -0.552      0.581      -0.108       0.061
C(methods)[T.fc]                           0.0043      0.043      0.101      0.919      -0.080       0.089
C(methods)[T.svm]                         -0.0946      0.043     -2.204      0.028      -0.179      -0.010
C(time_budget)[T.30]:C(methods)[T.DT]      0.0078      0.061      0.129      0.898      -0.111       0.127
C(time_budget)[T.60]:C(methods)[T.DT]      0.0049      0.061      0.081      0.935      -0.114       0.124
C(time_budget)[T.30]:C(methods)[T.RF]      0.0273      0.061      0.450      0.653      -0.092       0.146
C(time_budget)[T.60]:C(methods)[T.RF]      0.0209      0.061      0.344      0.731      -0.098       0.140
C(time_budget)[T.30]:C(methods)[T.fc]      0.0053      0.061      0.087      0.931      -0.114       0.124
C(time_budget)[T.60]:C(methods)[T.fc]      0.0009      0.061      0.015      0.988      -0.118       0.120
C(time_budget)[T.30]:C(methods)[T.svm]     0.0140      0.061      0.230      0.818      -0.105       0.133
C(time_budget)[T.60]:C(methods)[T.svm]     0.0174      0.061      0.286      0.775      -0.102       0.137
==============================================================================
Omnibus:                      247.044   Durbin-Watson:                   0.285
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              650.000
Skew:                          -1.779   Prob(JB):                    7.15e-142
Kurtosis:                       6.002   Cond. No.                         21.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
res

,sum_sq,df,F,PR(>F)
C(time_budget),0.024277,2.0,0.274496,0.760036
C(methods),0.774302,4.0,4.377454,0.001675
C(time_budget):C(methods),0.013954,8.0,0.039444,0.999977
Residual,31.175814,705.0,NaN,NaN


In [24]:
# Fits the model
model2 = ols('f1score ~ C(time_budget)+ C(methods)', df).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

Overall model F( 6, 713) =  3.043, p =  0.0060


In [25]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     3.043
Date:                Sun, 03 Nov 2019   Prob (F-statistic):            0.00604
Time:                        17:08:42   Log-Likelihood:                 108.46
No. Observations:                 720   AIC:                            -202.9
Df Residuals:                     713   BIC:                            -170.9
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.8189      0.021     39.711      0.000       0.778       0.859
C(time_budget)[T.30]     0.0142      0.019      0.745      0.457      -0.023       0.052
C(time_budget)[T.60]     0.0071      0.019      0.373      0.709      -0.030       0.045
C(methods)[T.DT]        -0.0262      0.025     -1.064      0.288      -0.075       0.022
C(methods)[T.RF]        -0.0076      0.025     -0.309      0.757      -0.056       0.041
C(methods)[T.fc]         0.0064      0.025      0.259      0.795      -0.042       0.055
C(methods)[T.svm]       -0.0842      0.025     -3.414      0.001      -0.133      -0.036
==============================================================================
Omnibus:                      248.210   Durbin-Watson:                   0.284
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              657.201
Skew:                          -1.784   Prob(JB):                    1.95e-143
Kurtosis:                       6.030   Cond. No.                         6.45
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

,sum_sq,df,F,PR(>F)
C(time_budget),0.024277,2.0,0.277486,0.757768
C(methods),0.774302,4.0,4.425147,0.001539
Residual,31.189768,713.0,NaN,NaN


In [27]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(time_budget),0.024277,0.012138,2.0,0.277486,0.757768,0.000759,-0.001973
C(methods),0.774302,0.193575,4.0,4.425147,0.001539,0.024206,0.018710
Residual,31.189768,0.043744,713.0,NaN,NaN,NaN,NaN


In [28]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    3c     DT  -0.0262 0.8011 -0.0936  0.0411  False
    3c     RF  -0.0076    0.9  -0.075  0.0597  False
    3c     fc   0.0064    0.9 -0.0609  0.0737  False
    3c    svm  -0.0842  0.006 -0.1515 -0.0168   True
    DT     RF   0.0186    0.9 -0.0487  0.0859  False
    DT     fc   0.0326 0.6539 -0.0347  0.0999  False
    DT    svm  -0.0579 0.1295 -0.1253  0.0094  False
    RF     fc    0.014    0.9 -0.0533  0.0814  False
    RF    svm  -0.0765 0.0167 -0.1439 -0.0092   True
    fc    svm  -0.0905 0.0023 -0.1579 -0.0232   True
----------------------------------------------------
